# **6N - Compiladores**
# **Projeto - Mini Pascal**
# **41829611 - Gabriel Coelho da Cunha**
# **41830075 - Renan de Oliveira Rocha**

## **Configuração**

In [1]:
!pip install rply

from rply import LexerGenerator, ParserGenerator
from rply.token import BaseBox
from re import match
from networkx import Graph, draw_networkx, coloring
from matplotlib.pyplot import show
from itertools import combinations

## **Gramática**

\<program> ::= program \<identifier> ; \<block> .

\<block> ::= \<variable declaration part> { \<subroutine declaration part> } \<statement part>

\<variable declaration part> ::= \<empty> |
var \<variable declaration> ;
    { \<variable declaration> ; }

\<variable declaration> ::= \<identifier> { , \<identifier> } : \<type>

\<type> ::= \<simple type> | \<array type>

\<array type> ::= array [ \<index range> ] of \<simple type>

\<index range> ::= \<integer constant> .. \<integer constant>

\<simple type> ::= char | integer | boolean

\<subroutine declaration part> ::= \<procedure declaration | function declaration>

\<procedure declaration> ::= procedure \<identifier> \<formal parameters> ; \<block>

\<function declaration> ::= function \<identifier> \<formal parameters> : \<type> ; \<block>

\<formal parameters> ::= ( \<param section> )

\<param section> ::= \<variable declaration> { ; \<variable declaration> ; }

\<statement part> ::= \<compound statement>

\<compound statement> ::= begin \<statement> { ; \<statement> } end

\<statement> ::= \<simple statement> | \<structured statement>

\<simple statement> ::= \<assignment statement> | \<function_procedure statement> |
\<read statement> | \<write statement>

\<assignment statement> ::= \<variable> := \<expression>

\<function_procedure statement> ::= \<function_procedure identifier>
| \<variable> := \<function_procedure identifier>

\<function_procedure identifier> ::= \<identifier>

\<read statement> ::= read ( \<variable> { , \<variable> } )

\<write statement> ::= write ( \<variable> { , \<variable> } )

\<structured statement> ::= \<compound statement> | \<if statement> |
<while statement>

\<if statement> ::= if \<expression> then \<statement> |
if \<expression> then \<statement> else \<statement>

\<while statement> ::= while \<expression> do \<statement>

\<expression> ::= \<simple expression> |
\<simple expression> \<relational operator> \<simple expression>

\<simple expression> ::= \<sign> \<term> { \<adding operator> \<term> }

\<term> ::= \<factor> { \<multiplying operator> \<factor> }

\<factor> ::= \<variable> | \<constant> | ( \<expression> ) | not \<factor>

\<relational operator> ::= = | \<> | \< | \<= | \>= | \> | or | and

\<sign> ::= + | - | \<empty>

\<adding operator> ::= + | -

\<multiplying operator> ::= * | div

\<variable> ::= \<entire variable> | \<indexed variable>

\<indexed variable> ::= \<array variable> [ \<expression> ]

\<array variable> ::= \<entire variable>

\<entire variable> ::= \<variable identifier>

\<variable identifier> ::= \<identifier>

## **Analisador Léxico**

In [2]:
lg = LexerGenerator()

# <special symbol> ::= +
lg.add("ADDITION", r"\+")
# <special symbol> ::= -
lg.add("SUBTRACTION", r"\-")
# <special symbol> ::= *
lg.add("MULTIPLICATION", r"\*")
# <special symbol> ::= =
lg.add("EQUALITY", r"\=")
# <special symbol> ::= <>
lg.add("INEQUALITY", r"\<\>")
# <special symbol> ::= <
lg.add("LESS_THAN", r"\<")
# <special symbol> ::= >
lg.add("GREATER_THAN", r"\>")
# <special symbol> ::= <=
lg.add("LESS_THAN_OR_EQUAL_TO", r"\<\=")
# <special symbol> ::= >=
lg.add("GREATER_THAN_OR_EQUAL_TO", r"\>\=")
# <special symbol> ::= (
lg.add("OPEN_PARENTHESES", r"\(")
# <special symbol> ::= )
lg.add("CLOSE_PARENTHESES", r"\)")
# <special symbol> ::= [
lg.add("OPEN_BRACKETS", r"\[")
# <special symbol> ::= ]
lg.add("CLOSE_BRACKETS", r"\]")
# <special symbol> ::= :=
lg.add("ASSIGNMENT", r"\:\=")
# <special symbol> ::= ..
lg.add("RANGE", r"\.\.")
# <special symbol> ::= .
lg.add("DOT", r"\.")
# <special symbol> ::= ,
lg.add("COMMA", r"\,")
# <special symbol> ::= ;
lg.add("SEMICOLON", r"\;")
# <special symbol> ::= :
lg.add("COLON", r"\:")
# <special symbol> ::= div
lg.add("DIVISION", "div")
# <special symbol> ::= or
lg.add("OR", "or")
# <special symbol> ::= and
lg.add("AND", "and")
# <special symbol> ::= not
lg.add("NOT", "not")
# <special symbol> ::= if
lg.add("IF", "if")
# <special symbol> ::= then
lg.add("THEN", "then")
# <special symbol> ::= else
lg.add("ELSE", "else")
# <special symbol> ::= of
lg.add("OF", "of")
# <special symbol> ::= while
lg.add("WHILE", "while")
# <special symbol> ::= do
lg.add("DO", "do")
# <special symbol> ::= begin
lg.add("BEGIN", "begin")
# <special symbol> ::= end
lg.add("END", "end")
# <special symbol> ::= read
lg.add("READ", "read")
# <special symbol> ::= write
lg.add("WRITE", "write")
# <special symbol> ::= var
lg.add("VAR", "var")
# <special symbol> ::= array
lg.add("ARRAY", "array")
# <special symbol> ::= function
lg.add("FUNCTION", "function")
# <special symbol> ::= procedure
lg.add("PROCEDURE", "procedure")
# <special symbol> ::= program
lg.add("PROGRAM", "program")
# <special symbol> ::= true
lg.add("TRUE", "true")
# <special symbol> ::= false
lg.add("FALSE", "false")
# <special symbol> ::= char
lg.add("CHAR", "char")
# <special symbol> ::= integer
lg.add("INTEGER", "integer")
# <special symbol> ::= boolean
lg.add("BOOLEAN", "boolean")
# <identifier> ::= <letter> { <letter or digit> }
lg.add("IDENTIFIER", r"[a-zA-Z]\w*")
# <integer constant> ::= <digit> { <digit> }
lg.add("INTEGER_CONSTANT", r"\d+")
# <character constant> ::= ''<letter or digit> {<letter or digit>}''
lg.add("CHARACTER_CONSTANT_ONE_OR_MORE", r'\"\w+\"')
# <character constant> ::= '<letter or digit>'
lg.add("CHARACTER_CONSTANT_ONE", r"\'\w\'")
# <letter> ::= a | b | c | d | e | f | g | h | i | j | k | l | m | n | o |
#              p | q | r | s | t | u | v | w | x | y | z | A | B | C |
#              D | E | F | G | H | I | J | K | L | M | N | O | P |
#              Q | R | S | T | W | V | W | X | Y | Z
lg.add("LETTER", r"[a-zA-Z]")
# <digit> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
lg.add("DIGIT", r"\d")

lg.ignore(r'\s+')

lexer = lg.build()

## **Classe base para aceitar Visitors**

In [3]:
class Expr(BaseBox):
  def accept(self, visitor):
    method_name = 'visit_{}'.format(self.__class__.__name__.lower())
    visit = getattr(visitor, method_name)
    return visit(self)

## **Arvore Sintática**

In [4]:
# funcao auxiliar para imprimir sem pular para proxima linha
printInLine = lambda msg: print(msg, end='')

In [5]:
# <program> ::= program <identifier> ; <block> .
class Program(Expr):
  def __init__(self, identifier, block):
    self.identifier = identifier
    self.block = block
  
  def print(self):
    printInLine("program ")
    printInLine(self.identifier)
    print(" ;")
    self.block.print()
    print(".")

# <block> ::= <variable declaration part>
#             { <subroutine declaration part> }
#             <statement part>
class Block(Expr):
  def __init__(self, variable_declaration_part, subroutine_declaration_part,
               statement_part):
    self.variable_declaration_part = variable_declaration_part
    self.subroutine_declaration_part = subroutine_declaration_part
    self.statement_part = statement_part
  
  def print(self):
    self.variable_declaration_part.print()
    if self.subroutine_declaration_part:
      self.subroutine_declaration_part.print()
    self.statement_part.print()

# <variable declaration part> ::= var <variable declaration> ;
#                                   { <variable declaration> ; }
class VariableDeclarationPart(Expr):
  def __init__(self, variable_declaration, variable_declaration_list):
    self.variable_declaration = variable_declaration
    self.variable_declaration_list = variable_declaration_list
  
  def print(self):
    printInLine("var ")
    self.variable_declaration.print()
    print(" ;")
    if self.variable_declaration_list:
      for variable_declaration in self.variable_declaration_list:
        variable_declaration.print()
        print(" ;")

# <variable declaration> ::= <identifier> { , <identifier> } : <type>
class VariableDeclaration(Expr):
  def __init__(self, identifier, identifier_list, type_):
    self.identifier = identifier
    self.identifier_list = identifier_list
    self.type_ = type_
  
  def print(self):
    printInLine(self.identifier)
    if self.identifier_list:
      for identifier in self.identifier_list:
        printInLine(", ")
        printInLine(identifier)
    printInLine(" : ")
    self.type_.print()

# <type> ::= <simple type> | <array type>
class Type(Expr):
  def __init__(self, type_):
    self.type_ = type_

  def print(self):
    self.type_.print()

# <array type> ::= array [ <index range> ] of <simple type>
class ArrayType(Expr):
  def __init__(self, index_range, simple_type):
    self.index_range = index_range
    self.simple_type = simple_type

  def print(self):
    printInLine("array [ ")
    self.index_range.print()
    printInLine(" ] of")
    self.simple_type.print()

# <index range> ::= <integer constant> .. <integer constant>
class IndexRange(Expr):
  def __init__(self, start, end):
    self.start = start
    self.end = end

  def print(self):
    printInLine(self.start)
    printInLine(" .. ")
    printInLine(self.end)

# <subroutine declaration part> ::= <procedure declaration> | <function declaration>
class SubroutineDeclarationPart(Expr):
  def __init__(self, declaration):
    self.declaration = declaration

  def print(self):
    self.declaration.print()

# <procedure declaration> ::= procedure <identifier> <formal parameters> ; <block>
class ProcedureDeclaration(Expr):
  def __init__(self, identifier, formal_parameters, block):
    self.identifier = identifier
    self.formal_parameters = formal_parameters
    self.block = block

  def print(self):
    printInLine("procedure ")
    printInLine(f"{self.identifier} ")
    self.formal_parameters.print()
    print(" ;")
    self.block.print()

# <function declaration> ::= function <identifier> <formal parameters> : <type> ; <block>
class FunctionDeclaration(Expr):
  def __init__(self, identifier, formal_parameters, type_, block):
    self.identifier = identifier
    self.formal_parameters = formal_parameters
    self.type_ = type_
    self.block = block

  def print(self):
    printInLine("function ")
    printInLine(f"{self.identifier} ")
    self.formal_parameters.print()
    printInLine(" : ")
    self.type_.print()
    print(" ;")
    self.block.print()

# <formal parameters> ::= ( <param section> )
class FormalParameters(Expr):
  def __init__(self, param_section):
    self.param_section = param_section

  def print(self):
    printInLine("( ")
    self.param_section.print()
    printInLine(" )")

# <param section> ::= <variable declaration> { ; <variable declaration> ; }
class ParamSection(Expr):
  def __init__(self, variable_declaration, variable_declaration_list):
    self.variable_declaration = variable_declaration
    self.variable_declaration_list = variable_declaration_list

  def print(self):
    self.variable_declaration.print()
    if self.variable_declaration_list:
      print(" ;")
      for variable_declaration in self.variable_declaration_list:
        variable_declaration.print()
        print(" ;")

# <statement part> ::= <compound statement>
class StatementPart(Expr):
  def __init__(self, compound_statement):
    self.compound_statement = compound_statement

  def print(self):
    self.compound_statement.print()

# <compound statement> ::= begin <statement> { ; <statement> } end
class CompoundStatement(Expr):
  def __init__(self, statement, statement_list):
    self.statement = statement
    self.statement_list = statement_list

  def print(self):
    print("begin")
    self.statement.print()
    if self.statement_list:
      for statement in self.statement_list:
        print(" ;")
        statement.print()
    print()
    print("end")

# <statement> ::= <simple statement> | <structured statement>
# <simple statement> ::= <assignment statement> |
#                        <function_procedure statement> |
#                        <read statement> |
#                        <write statement>
# <structured statement> ::= <compound statement> | <if statement> |
#                            <while statement>
class Statement(Expr):
  def __init__(self, statement):
    self.statement = statement

  def print(self):
    self.statement.print()

# <assignment statement> ::= <variable> := <expression>
class AssignmentStatement(Expr):
  def __init__(self, variable, expression):
    self.variable = variable
    self.expression = expression

  def print(self):
    self.variable.print()
    printInLine(" := ")
    self.expression.print()

# <function_procedure statement> ::= <function_procedure identifier> |
#                                    <variable> := <function_procedure identifier>
class FunctionProcedureStatement(Expr):
  def __init__(self, function_procedure_identifier, variable):
    self.function_procedure_identifier = function_procedure_identifier
    self.variable = variable

  def print(self):
    if self.variable:
      self.variable.print()
      printInLine(" := ")
    self.function_procedure_identifier.print()

# <read statement> ::= read ( <variable> { , <variable> } )
class ReadStatement(Expr):
  def __init__(self, variable, variable_list):
    self.variable = variable
    self.variable_list = variable_list

  def print(self):
    printInLine("read ( ")
    self.variable.print()
    if self.variable_list:
      for variable in self.variable_list:
        printInLine(", ")
        variable.print()
    printInLine(" )")

# <write statement> ::= write ( <variable> { , <variable> } )
class WriteStatement(Expr):
  def __init__(self, variable, variable_list):
    self.variable = variable
    self.variable_list = variable_list

  def print(self):
    printInLine("write ( ")
    self.variable.print()
    if self.variable_list:
      for variable in self.variable_list:
        printInLine(", ")
        variable.print()
    printInLine(" )")

# <if statement> ::= if <expression> then <statement> |
#                    if <expression> then <statement> else <statement>
class IfStatement(Expr):
  def __init__(self, expression, statement, else_statement):
    self.expression = expression
    self.statement = statement
    self.else_statement = else_statement

  def print(self):
    printInLine("if ")
    self.expression.print()
    print(" then")
    self.statement.print()
    if self.else_statement:
      print()
      print("else")
      self.else_statement.print()

# <while statement> ::= while <expression> do <statement>
class WhileStatement(Expr):
  def __init__(self, expression, statement):
    self.expression = expression
    self.statement = statement

  def print(self):
    printInLine("while ")
    self.expression.print()
    print(" do")
    self.statement.print()

# <expression> ::= <simple expression> |
#                  <simple expression> <relational operator> <simple expression>
class Expression(Expr):
  def __init__(self, left, relational_operator, right):
    self.left = left
    self.relational_operator = relational_operator
    self.right = right

  def print(self):
    self.left.print()
    if self.relational_operator and self.right:
      printInLine(" ")
      self.relational_operator.print()
      printInLine(" ")
      self.right.print()

# <simple expression> ::= <sign> <term> { <adding operator> <term> }
class SimpleExpression(Expr):
  def __init__(self, sign, left, adding_operator, right):
    self.sign = sign
    self.left = left
    self.adding_operator = adding_operator
    self.right = right

  def print(self):
    if self.sign:
      self.sign.print()
      printInLine(" ")
    self.left.print()
    if self.adding_operator and self.right:
      printInLine(" ")
      self.adding_operator.print()
      printInLine(" ")
      self.right.print()

# <term> ::= <factor> { <multiplying operator> <factor> }
class Term(Expr):
  def __init__(self, left, multiplying_operator, right):
    self.left = left
    self.multiplying_operator = multiplying_operator
    self.right = right

  def print(self):
    self.left.print()
    if self.multiplying_operator and self.right:
      printInLine(" ")
      self.multiplying_operator.print()
      printInLine(" ")
      self.right.print()

# <factor> ::= <variable> | <constant> | ( <expression> ) | not <factor>
class Factor(Expr):
  def __init__(self, factor, has_parentheses=False, has_not=False):
    self.factor = factor
    self.has_parentheses = has_parentheses
    self.has_not = has_not

  def print(self):
    if self.has_parentheses:
      printInLine("( ")
    elif self.has_not:
      printInLine("not ")
    self.factor.print()
    if self.has_parentheses:
      printInLine(" )")

# <indexed variable> ::= <array variable> [ <expression> ]
class IndexedVariable(Expr):
  def __init__(self, array_variable, expression):
    self.array_variable = array_variable
    self.expression = expression

  def print(self):
    self.array_variable.print()
    printInLine(" [ ")
    self.expression.print()
    printInLine(" ]")

# <simple type> ::= char | integer | boolean
# <function_procedure identifier> ::= <identifier>
# <relational operator> ::= = | <> | < | <= | >= | > | or | and
# <sign> ::= + | -
# <adding operator> ::= + | -
# <multiplying operator> ::= * | div
# <variable identifier> ::= <identifier>
class SingleValue(Expr):
  def __init__(self, value):
    self.value = value

  def print(self):
    printInLine(self.value)

# <variable declaration part> ::= <empty>
# <sign> ::= <empty>
class Empty(Expr):
  def print(self):
    pass

## **Analisador Sintático**

In [6]:
pg = ParserGenerator(
    [
     "ADDITION", "SUBTRACTION", "MULTIPLICATION", "EQUALITY", "INEQUALITY",
     "LESS_THAN", "GREATER_THAN", "LESS_THAN_OR_EQUAL_TO",
     "GREATER_THAN_OR_EQUAL_TO", "OPEN_PARENTHESES", "CLOSE_PARENTHESES",
     "OPEN_BRACKETS", "CLOSE_BRACKETS", "ASSIGNMENT", "DOT", "COMMA",
     "SEMICOLON", "COLON", "RANGE", "DIVISION", "OR", "AND", "NOT", "IF", "THEN",
     "ELSE", "OF", "WHILE", "DO", "BEGIN", "END", "READ",
     "WRITE", "VAR", "ARRAY", "FUNCTION", "PROCEDURE",
     "PROGRAM", "TRUE", "FALSE", "CHAR", "INTEGER", "BOOLEAN",
     "IDENTIFIER", "INTEGER_CONSTANT", "CHARACTER_CONSTANT_ONE_OR_MORE",
     "CHARACTER_CONSTANT_ONE", "LETTER", "DIGIT",
    ],
    precedence=[
        ("left", ["ADDITION", "SUBTRACTION"]),
        ("left", ["MULTIPLICATION", "DIVISION"]),
    ]
)

@pg.error
def error_handler(token):
  value = token.getstr()
  position = token.getsourcepos()

  raise ValueError(f"Token '{value}' wasn't expected at line '{position.lineno}' and column '{position.colno}'.")

# <program> ::= program <identifier> ; <block> .
@pg.production("program : PROGRAM IDENTIFIER SEMICOLON block DOT")
def program(p):
  return Program(p[1].getstr(), p[3])

# <block> ::= <variable declaration part>
#             { <subroutine declaration part> }
#             <statement part>
@pg.production("block : variable_declaration_part statement_part")
@pg.production("""
  block : variable_declaration_part
          subroutine_declaration_part
          statement_part
""")
def block(p):
  if len(p) == 2:
    return Block(p[0], None, p[1])
  elif len(p) == 3:
    return Block(p[0], p[1], p[2])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <variable declaration part> ::= var <variable declaration> ;
#                                   { <variable declaration> ; }
@pg.production("variable_declaration_part : VAR variable_declaration SEMICOLON")
@pg.production("variable_declaration_part : VAR variable_declaration SEMICOLON variable_declaration_list")
def variable_declaration_part(p):
  if len(p) == 3:
    return VariableDeclarationPart(p[1], None)
  elif len(p) == 4:
    return VariableDeclarationPart(p[1], p[3])
  else:
    raise AssertionError("Sintaxe incorreta!")

@pg.production("variable_declaration_list : ")
@pg.production("variable_declaration_list : variable_declaration SEMICOLON")
@pg.production("""
  variable_declaration_list : variable_declaration SEMICOLON
                              variable_declaration_list
""")
def variable_declaration_list(p):
  if not p:
    return []
  elif len(p) == 2:
    return [p[0]]
  elif len(p) == 3:
    return [p[0]] + p[2]
  else:
    raise AssertionError("Sintaxe incorreta!")

# <variable declaration> ::= <identifier> { , <identifier> } : <type>
@pg.production("variable_declaration : IDENTIFIER COLON type")
@pg.production("variable_declaration : IDENTIFIER COMMA identifier_list COLON type")
def variable_declaration(p):
  if len(p) == 3:
    return VariableDeclaration(p[0].getstr(), None, p[2])
  elif len(p) == 5:
    return VariableDeclaration(p[0].getstr(), p[2], p[4])
  else:
    raise AssertionError("Sintaxe incorreta!")

@pg.production("identifier_list : ")
@pg.production("identifier_list : IDENTIFIER")
@pg.production("identifier_list : IDENTIFIER COMMA identifier_list")
def list_2(p):
  if not p:
    return []
  elif len(p) == 1:
    return [p[0].getstr()]
  elif len(p) == 3:
    return [p[0].getstr()] + p[2]
  else:
    raise AssertionError("Sintaxe incorreta!")

# <type> ::= <simple type> | <array type>
@pg.production("type : simple_type")
@pg.production("type : array_type")
def variable_type(p):
  return Type(p[0])

# <array type> ::= array [ <index range> ] of <simple type>
@pg.production("array_type : ARRAY OPEN_BRACKETS index_range CLOSE_BRACKETS OF simple_type")
def array_type(p):
  return ArrayType(p[2], p[5])

# <index range> ::= <integer constant> .. <integer constant>
@pg.production("index_range : INTEGER_CONSTANT RANGE INTEGER_CONSTANT")
def index_range(p):
  return IndexRange(p[0].getstr(), p[2].getstr())

# <subroutine declaration part> ::= <procedure declaration | function declaration>
@pg.production("subroutine_declaration_part : procedure_declaration")
@pg.production("subroutine_declaration_part : function_declaration")
def subroutine_declaration_part(p):
  return SubroutineDeclarationPart(p[0])

# <procedure declaration> ::= procedure <identifier> <formal parameters> ; <block>
@pg.production("""
  procedure_declaration : PROCEDURE IDENTIFIER formal_parameters SEMICOLON
                          block
""")
def procedure_declaration(p):
  return ProcedureDeclaration(p[1].getstr(), p[2], p[4])

# <function declaration> ::= function <identifier> <formal parameters> : <type> ; <block>
@pg.production("""
  function_declaration : FUNCTION IDENTIFIER formal_parameters COLON type 
                          SEMICOLON block
""")
def function_declaration(p):
  return FunctionDeclaration(p[1].getstr(), p[2], p[4], p[6])

# <formal parameters> ::= ( <param section> )
@pg.production("formal_parameters : OPEN_PARENTHESES param_section CLOSE_PARENTHESES")
def formal_parameters(p):
  return FormalParameters(p[1])

# <param section> ::= <variable declaration> { ; <variable declaration> ; }
@pg.production("param_section : variable_declaration")
@pg.production("param_section : variable_declaration SEMICOLON variable_declaration_list")
def param_section(p):
  if len(p) == 1:
    return ParamSection(p[0], None)
  elif len(p) == 3:
    return ParamSection(p[0], p[2])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <statement part> ::= <compound statement>
@pg.production("statement_part : compound_statement")
def statement_part(p):
  return StatementPart(p[0])

# <compound statement> ::= begin <statement> { ; <statement> } end
@pg.production("compound_statement : BEGIN statement END")
@pg.production("compound_statement : BEGIN statement SEMICOLON statement_list END")
def compound_statement(p):
  if len(p) == 3:
    return CompoundStatement(p[1], None)
  elif len(p) == 5:
    return CompoundStatement(p[1], p[3])
  else:
    raise AssertionError("Sintaxe incorreta!")

@pg.production("statement_list : ")
@pg.production("statement_list : statement")
@pg.production("statement_list : statement SEMICOLON statement_list")
def statement_list(p):
  if not p:
    return []
  elif len(p) == 1:
    return [p[0]]
  elif len(p) == 3:
    return [p[0]] + p[2]
  else:
    raise AssertionError("Sintaxe incorreta!")

# <statement> ::= <simple statement> | <structured statement>
# <simple statement> ::= <assignment statement> | <function_procedure statement> |
#                        <read statement> | <write statement>
# <structured statement> ::= <compound statement> | <if statement> |
#                            <while statement>
@pg.production("statement : simple_statement")
@pg.production("statement : structured_statement")
@pg.production("simple_statement : assignment_statement")
@pg.production("simple_statement : function_procedure_statement")
@pg.production("simple_statement : read_statement")
@pg.production("simple_statement : write_statement")
@pg.production("structured_statement : compound_statement")
@pg.production("structured_statement : if_statement")
@pg.production("structured_statement : while_statement")
def statement(p):
  return Statement(p[0])

# <assignment statement> ::= <variable> := <expression>
@pg.production("assignment_statement : variable ASSIGNMENT expression")
def assignment_statement(p):
  return AssignmentStatement(p[0], p[2])

# <function_procedure statement> ::= <function_procedure identifier> |
#                                    <variable> := <function_procedure identifier>
@pg.production("function_procedure_statement : function_procedure_identifier")
@pg.production("function_procedure_statement : variable ASSIGNMENT function_procedure_identifier")
def function_procedure_statement(p):
  if len(p) == 1:
    return FunctionProcedureStatement(p[0], None)
  elif len(p) == 3:
    return FunctionProcedureStatement(p[2], p[0])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <read statement> ::= read ( <variable> { , <variable> } )
@pg.production("read_statement : READ OPEN_PARENTHESES variable CLOSE_PARENTHESES")
@pg.production("read_statement : READ OPEN_PARENTHESES variable COMMA variable_list CLOSE_PARENTHESES")
def read_statement(p):
  if len(p) == 4:
    return ReadStatement(p[2], None)
  elif len(p) == 6:
    return ReadStatement(p[2], p[4])
  else:
    raise AssertionError("Sintaxe incorreta!")

@pg.production("variable_list : ")
@pg.production("variable_list : variable")
@pg.production("variable_list : variable COMMA variable_list")
def variable_list(p):
  if not p:
    return []
  elif len(p) == 1:
    return [p[0]]
  elif len(p) == 3:
    return [p[0]] + p[2]
  else:
    raise AssertionError("Sintaxe incorreta!")

# <write statement> ::= write ( <variable> { , <variable> } )
@pg.production("write_statement : WRITE OPEN_PARENTHESES variable CLOSE_PARENTHESES")
@pg.production("write_statement : WRITE OPEN_PARENTHESES variable COMMA variable_list CLOSE_PARENTHESES")
def write_statement(p):
  if len(p) == 4:
    return WriteStatement(p[2], None)
  elif len(p) == 6:
    return WriteStatement(p[2], p[4])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <if statement> ::= if <expression> then <statement> |
#                    if <expression> then <statement> else <statement>
@pg.production("if_statement : IF expression THEN statement")
@pg.production("if_statement : IF expression THEN statement ELSE statement")
def if_statement(p):
  if len(p) == 4:
    return IfStatement(p[1], p[3], None)
  elif len(p) == 6:
    return IfStatement(p[1], p[3], p[5])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <while statement> ::= while <expression> do <statement>
@pg.production("while_statement : WHILE expression DO statement")
def while_statement(p):
  return WhileStatement(p[1], p[3])

# <expression> ::= <simple expression> |
#                  <simple expression> <relational operator> <simple expression>
@pg.production("expression : simple_expression")
@pg.production("expression : simple_expression relational_operator simple_expression")
def expression(p):
  if len(p) == 1:
    return Expression(p[0], None, None)
  elif len(p) == 3:
    return Expression(p[0], p[1], p[2])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <simple expression> ::= <sign> <term> { <adding operator> <term> }
@pg.production("simple_expression : term")
@pg.production("simple_expression : sign term")
@pg.production("simple_expression : term adding_operator term")
@pg.production("simple_expression : sign term adding_operator term")
def simple_expression(p):
  if len(p) == 1:
    return SimpleExpression(None, p[0], None, None)
  elif len(p) == 2:
    return SimpleExpression(p[0], p[1], None, None)
  elif len(p) == 3:
    return SimpleExpression(None, p[0], p[1], p[2])
  elif len(p) == 4:
    return SimpleExpression(p[0], p[1], p[2], p[3])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <term> ::= <factor> { <multiplying operator> <factor> }
@pg.production("term : factor")
@pg.production("term : factor multiplying_operator factor")
def term(p):
  if len(p) == 1:
    return Term(p[0], None, None)
  elif len(p) == 3:
    return Term(p[0], p[1], p[2])
  else:
    raise AssertionError("Sintaxe incorreta!")

# <factor> ::= <variable> | <constant> | ( <expression> ) | not <factor>
@pg.production("factor : variable")
@pg.production("factor : constant")
@pg.production("factor : OPEN_PARENTHESES expression CLOSE_PARENTHESES")
@pg.production("factor : NOT factor")
def factor(p):
  if len(p) == 1:
    return Factor(p[0])
  elif len(p) == 2:
    return Factor(p[1], has_not=True)
  elif len(p) == 3:
    return Factor(p[1], has_parentheses=True)
  else:
    raise AssertionError("Sintaxe incorreta!")

# <variable> ::= <entire variable> | <indexed variable>
# <array variable> ::= <entire variable>
# <entire variable> ::= <variable indetifier>
@pg.production("variable : entire_variable")
@pg.production("variable : indexed_variable")
@pg.production("array_variable : entire_variable")
@pg.production("entire_variable : variable_identifier")
def variable(p):
  return p[0]

# <indexed variable> ::= <array variable> [ <expression> ]
@pg.production("indexed_variable : array_variable OPEN_BRACKETS expression CLOSE_BRACKETS")
def indexed_variable(p):
  return IndexedVariable(p[0], p[2])

# <simple type> ::= char | integer | boolean
# <function_procedure identifier> ::= <identifier>
# <relational operator> ::= = | <> | < | <= | >= | > | or | and
# <sign> ::= + | -
# <adding operator> ::= + | -
# <multiplying operator> ::= * | div
# <variable identifier> ::= <identifier>
# <constant>
@pg.production("simple_type : CHAR")
@pg.production("simple_type : INTEGER")
@pg.production("simple_type : BOOLEAN")
@pg.production("function_procedure_identifier : IDENTIFIER")
@pg.production("relational_operator : EQUALITY")
@pg.production("relational_operator : INEQUALITY")
@pg.production("relational_operator : LESS_THAN")
@pg.production("relational_operator : LESS_THAN_OR_EQUAL_TO")
@pg.production("relational_operator : GREATER_THAN_OR_EQUAL_TO")
@pg.production("relational_operator : GREATER_THAN")
@pg.production("relational_operator : OR")
@pg.production("relational_operator : AND")
@pg.production("sign : ADDITION")
@pg.production("sign : SUBTRACTION")
@pg.production("adding_operator : ADDITION")
@pg.production("adding_operator : SUBTRACTION")
@pg.production("multiplying_operator : MULTIPLICATION")
@pg.production("multiplying_operator : DIVISION")
@pg.production("variable_identifier : IDENTIFIER")
@pg.production("constant : INTEGER_CONSTANT")
@pg.production("constant : CHARACTER_CONSTANT_ONE")
@pg.production("constant : CHARACTER_CONSTANT_ONE_OR_MORE")
@pg.production("constant : IDENTIFIER")
@pg.production("constant : TRUE")
@pg.production("constant : FALSE")
def single_value(p):
  return SingleValue(p[0].getstr())

# <variable declaration> ::= <empty>
# <sign> ::= <empty>
@pg.production("variable_declaration_part : ")
@pg.production("sign : ")
def empty(p):
  return Empty()

parser = pg.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:383: ParserGeneratorWarning: Token 'LETTER' is unused
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:383: ParserGeneratorWarning: Token 'DIGIT' is unused
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:383: ParserGeneratorWarning: 49 shift/reduce conflicts
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:383: ParserGeneratorWarning: 44 reduce/reduce conflicts


## **Visitor base**

In [7]:
class Visitor(Expr):
  pass

## **Visitor para montar tabela de simbolos**

In [8]:
ST = {}

class SymbolTable(Visitor):
  def visit_program(self, p):
    p.block.accept(self)

  def visit_block(self, b):
    b.variable_declaration_part.accept(self)
    if b.subroutine_declaration_part:
      b.subroutine_declaration_part.accept(self)

  def visit_variabledeclarationpart(self, vdp):
    vdp.variable_declaration.accept(self)
    if vdp.variable_declaration_list:
      for v in vdp.variable_declaration_list: 
        v.accept(self)

  def visit_subroutinedeclarationpart(self, sdp):
    sdp.declaration.accept(self)

  def visit_declaration(self, d):
    d.block.accept(self)

  def visit_proceduredeclaration(self, pd):
    self.visit_declaration(pd)

  def annotate_symbol(self, node, identifier):
    if identifier not in ST:
      ST[identifier] = node.type_.accept(self)
    else:
      raise AssertionError(f"{identifier} is already declared")

  def visit_functiondeclaration(self, fd):
    self.annotate_symbol(fd, fd.identifier)
    self.visit_declaration(fd)

  def visit_variabledeclaration(self, vd):
    self.annotate_symbol(vd, vd.identifier)
    if vd.identifier_list:
      for i in vd.identifier_list:
        self.annotate_symbol(vd, i)

  def visit_type(self, t):
    return t.type_.accept(self)

  def visit_arraytype(self, at):
    return f"array of {at.simple_type.accept(self)}"

  def visit_singlevalue(self, sv):
    return sv.value

  def visit_empty(self, e):
    pass

## **Visitor para decorar arvore sintatica**

In [9]:
class Decorator(Visitor):
  def visit_program(self, p):
    p.block.accept(self)

  def visit_block(self, b):
    if b.subroutine_declaration_part:
      b.subroutine_declaration_part.accept(self)
    b.statement_part.accept(self)

  def visit_subroutinedeclarationpart(self, sdp):
    sdp.declaration.accept(self)

  def declaration(self, d):
    d.block.accept(self)

  def visit_proceduredeclaration(self, pd):
    self.declaration(pd)

  def visit_functiondeclaration(self, fd):
    fd.decor_type = ST[fd.identifier]
    self.declaration(fd)

  def visit_statementpart(self, sp):
    sp.compound_statement.accept(self)

  def visit_compoundstatement(self, cs):
    cs.statement.accept(self)
    if cs.statement_list:
      for s in cs.statement_list:
        s.accept(self)

  def visit_statement(self, s):
    if type(s.statement).__name__ not in ("ReadStatement", "WriteStatement"):
      s.statement.accept(self)

  def visit_ifstatement(self, ifs):
    ifs.statement.accept(self)
    if ifs.else_statement:
      ifs.else_statement.accept(self)

  def visit_whilestatement(self, ws):
    ws.statement.accept(self)

  def decorate_node(self, n, left, right):
    if left.decor_type == right.decor_type:
      n.decor_type = left.decor_type
    else:
      n.decor_type = f"operation between {left.decor_type} and {right.decor_type}"

  def visit_assignmentstatement(self, ass):
    ass.variable.accept(self)
    ass.expression.accept(self)
    self.decorate_node(ass, ass.variable, ass.expression)

  def visit_functionprocedurestatement(self, fps):
    if fps.variable:
      fps.variable.accept(self)
      fps.funcion_procedure_identifier.accept(self)
      self.decorate_node(fps, fps.variable, fps.funcion_procedure_identifier)

  def visit_expression(self, e):
    if e.relational_operator:
      e.left.accept(self)
      e.relational_operator.accept(self)
      e.right.accept(self)
      self.decorate_node(e, e.left, e.right)
    else:
      e.left.accept(self)
      self.decorate_node(e, e.left, e.left)

  def visit_simpleexpression(self, se):
    if se.sign:
      se.sign.accept(self)
    if se.adding_operator:
      se.left.accept(self)
      se.adding_operator.accept(self)
      se.right.accept(self)
      self.decorate_node(se, se.left, se.right)
    else:
      se.left.accept(self)
      self.decorate_node(se, se.left, se.left)

  def visit_term(self, t):
    if t.multiplying_operator:
      t.left.accept(self)
      t.multiplying_operator.accept(self)
      t.right.accept(self)
      self.decorate_node(t, t.left, t.right)
    else:
      t.left.accept(self)
      self.decorate_node(t, t.left, t.left)

  def visit_factor(self, f):
    f.factor.accept(self)
    self.decorate_node(f, f.factor, f.factor)

  def decorate_variable(self, sv, tp):
    sv.decor_type = tp

  def integer(self, i):
    self.decorate_variable(i, "integer")

  def char(self, c):
    self.decorate_variable(c, "char")

  def boolean(self, b):
    self.decorate_variable(b, "boolean")

  def identifier(self, i):
    if i.value in ST:
      self.decorate_variable(i, ST[i.value])
    else:
      raise AssertionError(f"Id '{i.value}' not declared!")    

  def visit_singlevalue(self, sv):
    if match(r"\d+", sv.value):
      self.integer(sv)
    elif match(r"""(\'\w\')|(\"\w+\")""", sv.value):
      self.char(sv)
    elif match(r"true|false", sv.value):
      self.boolean(sv)
    elif not match(r"div", sv.value) and \
        match(r"[a-zA-Z]\w*", sv.value):
      self.identifier(sv)
    else:
      pass

  def visit_empty(self, e):
    pass

## **Visitor para verificar tipos**

In [10]:
class TypeVerifier(Visitor):
  def visit_program(self, p):
    p.block.accept(self)

  def visit_block(self, b):
    if b.subroutine_declaration_part:
      b.subroutine_declaration_part.accept(self)
    b.statement_part.accept(self)

  def visit_subroutinedeclarationpart(self, sdp):
    sdp.declaration.accept(self)

  def declaration(self, d):
    d.block.accept(self)

  def visit_proceduredeclaration(self, pd):
    self.declaration(pd)

  def visit_functiondeclaration(self, fd):
    self.declaration(fd)

  def visit_statementpart(self, sp):
    sp.compound_statement.accept(self)

  def visit_compoundstatement(self, cs):
    cs.statement.accept(self)
    if cs.statement_list:
      for s in cs.statement_list:
        s.accept(self)

  def visit_statement(self, s):
    if type(s.statement).__name__ not in ("ReadStatement", "WriteStatement"):
      s.statement.accept(self)

  def visit_ifstatement(self, ifs):
    ifs.statement.accept(self)
    if ifs.else_statement:
      ifs.else_statement.accept(self)

  def visit_whilestatement(self, ws):
    ws.statement.accept(self)

  def compare_nodes(self, left, right):
    if left.decor_type != right.decor_type:
      raise AssertionError(f"Type error: '{left.value}' type is '{left.decor_type}' and you are trying to assign a '{right.decor_type}'.")

  def visit_assignmentstatement(self, ass):
    ass.variable.accept(self)
    ass.expression.accept(self)
    self.compare_nodes(ass.variable, ass.expression)

  def visit_functionprocedurestatement(self, fps):
    if fps.variable:
      fps.variable.accept(self)
      fps.funcion_procedure_identifier.accept(self)
      self.compare_nodes(fps.variable, fps.funcion_procedure_identifier)

  def visit_expression(self, e):
    if e.relational_operator:
      e.left.accept(self)
      e.right.accept(self)
      self.compare_nodes(e.left, e.right)

  def visit_simpleexpression(self, se):
    if se.adding_operator:
      se.left.accept(self)
      se.right.accept(self)
      self.compare_nodes(se.left, se.right)

  def visit_term(self, t):
    if t.multiplying_operator:
      t.left.accept(self)
      t.right.accept(self)
      self.compare_nodes(t.left, t.right)

  def visit_singlevalue(self, sv):
    pass

## **Visitor para gerar codigo intermediario P-Code**

In [11]:
class IntermediateCode(Visitor):
  def __init__(self):
    self.oc = {
        "+": 2,
        "-": 3,
        "*": 4,
        "div": 5
    }
    self.ic = []

  def visit_program(self, p):
    p.block.accept(self)

  def visit_block(self, b):
    if b.subroutine_declaration_part:
      b.subroutine_declaration_part.accept(self)
    b.statement_part.accept(self)

  def visit_subroutinedeclarationpart(self, sdp):
    sdp.declaration.accept(self)

  def declaration(self, d):
    d.block.accept(self)

  def visit_proceduredeclaration(self, pd):
    self.declaration(pd)

  def visit_functiondeclaration(self, fd):
    self.declaration(fd)

  def visit_statementpart(self, sp):
    sp.compound_statement.accept(self)

  def visit_compoundstatement(self, cs):
    cs.statement.accept(self)
    if cs.statement_list:
      for s in cs.statement_list:
        s.accept(self)

  def visit_statement(self, s):
    if type(s.statement).__name__ not in ("ReadStatement", "WriteStatement"):
      s.statement.accept(self)

  def visit_ifstatement(self, ifs):
    ifs.statement.accept(self)
    if ifs.else_statement:
      ifs.else_statement.accept(self)

  def visit_whilestatement(self, ws):
    ws.statement.accept(self)

  def generate_ic_for_assignment(self, a):
    self.ic.append(f"sto 0 {a}")

  def visit_assignmentstatement(self, ass):
    if ass.decor_type == "integer":
      ass.expression.accept(self)
      self.generate_ic_for_assignment(ass.variable.value)

  def visit_functionprocedurestatement(self, fps):
    if fps.variable and fps.decor_type == "integer":
      fps.funcion_procedure_identifier.accept(self)
      self.generate_ic_for_assignment(fps.variable.value)

  def visit_expression(self, e):
    e.left.accept(self)

  def generate_ic_for_operation(self, o):
    self.ic.append(f"opr 0 {self.oc[o]}")

  def visit_simpleexpression(self, se):
    se.left.accept(self)
    if se.adding_operator:
      se.right.accept(self)
      self.generate_ic_for_operation(se.adding_operator.value)

  def visit_term(self, t):
    t.left.accept(self)
    if t.multiplying_operator:
      t.right.accept(self)
      self.generate_ic_for_operation(t.multiplying_operator.value)

  def visit_factor(self, f):
    f.factor.accept(self)

  def generate_ic_for_integer(self, i):
    self.ic.append(f"lit 0 {i}")

  def generate_ic_for_id(self, i):
    self.ic.append(f"lod 0 {i}")

  def visit_singlevalue(self, sv):
    if match(r"\d+", sv.value):
      self.generate_ic_for_integer(sv.value)
    elif not ( \
          match(r"""(\'\w\')|(\"\w+\")""", sv.value) or \
          match(r"true|false", sv.value) \
          ) and \
          not match(r"div", sv.value) and \
          match(r"[a-zA-Z]\w*", sv.value):
      self.generate_ic_for_id(sv.value)
    else:
      pass

  def visit_empty(self, e):
    pass

## **Alocacao de registradores para Intel 8086**

In [12]:
class RegisterAllocation:
  def __init__(self):
    self.ic_la = []
    self.edges = []
    self.graph = Graph()

  def liveness_analysis(self, ic):
    la = set()
    r = 0

    for i in ic:
      if i.startswith("lit") or i.startswith("lod"):
        la.add(r)
        r += 1
      elif i.startswith("opr") or i.startswith("sto"):
        la.remove(max(la))
        r -= 1
      
      if la:
        self.ic_la.append((i, la.copy()))
      else:
        self.ic_la.append((i, {}))

  def add_graph_edges(self):
    for _, la in self.ic_la:
      edges = combinations(la, 2)
      for a, b in edges:
        self.edges.append([a, b])
    self.graph.add_edges_from(self.edges)

  def visualize_graph(self):
    draw_networkx(self.graph)
    show()

## **Geracao de codigo final para Intel 8086**

In [13]:
class Intel8086Code:
  def __init__(self):
    self.graph_colors = {}
    self.operations = {
        2: "add",
        3: "sub",
        4: "mul",
        5: "div"
    }
    self.registers = ["ax","bx","cx","dx"]
    self.code = []

  def color_graph(self, graph):
    self.graph_colors = coloring.greedy_color(graph, strategy="largest_first")
    if not self.graph_colors:
      self.graph_colors = {0: 0}

  def generate_code(self, ic):
    r = 0
    for i in ic:
      ins, _, val = i.split()
      if ins == "lit" or ins == "lod":
        code = f"mov {self.registers[self.graph_colors[r]]}, {val}"
        r += 1
      elif ins == "opr":
        r -= 1
        o = self.operations[int(val)]
        r1 = self.registers[self.graph_colors[r-1]]
        r2 = self.registers[self.graph_colors[r]]
        code = f"{o} {r1}, {r2}"
      elif ins == "sto":
        r -= 1
        code = f"mov {val}, {self.registers[self.graph_colors[r]]}"
      
      self.code.append(code)

## **Teste 1**

In [14]:
test = """
          program conta_ate_n ;
            var n, n1, n2 : integer ;
            procedure contar ( n : integer ) ;
              var c : integer ;
                  msg : char ;
              begin
                c := 0 ;

                msg := "comecando" ;
                write ( msg ) ;

                while c < n do
                  c := c + 1 ;
                  write ( c ) ;

                msg := "terminando" ;
                write ( msg )
              end
            begin
              read ( n ) ;
              contar;
              n1 := 10;
              n2 := n1 + 5;
            end
          .
"""
tree = parser.parse(lexer.lex(test))
print("--------------- Symbol Table ----------------")
ST = {}
tree.accept(SymbolTable())
for k,v in ST.items():
  print(k, v)
print("---------------------------------------------")
print("----------------- Decorator -----------------")
tree.accept(Decorator())
print("---------------------------------------------")
print("--------------- Type Verifier ---------------")
tree.accept(TypeVerifier())
print("---------------------------------------------")
print("------------- Intermediate Code -------------")
ic = IntermediateCode()
tree.accept(ic)
ic = ic.ic
print("---------------------------------------------")
print("------------ Register Allocation ------------")
ra = RegisterAllocation()
ra.liveness_analysis(ic)
ra.add_graph_edges()
for i in ra.ic_la:
  print(i)
print("-------------- Intel 8086 Code --------------")
intel_8086 = Intel8086Code()
intel_8086.color_graph(ra.graph)
intel_8086.generate_code(ic)
for c in intel_8086.code:
  print(c)
print("---------------------------------------------")

--------------- Symbol Table ----------------
n integer
n1 integer
n2 integer
c integer
msg char
---------------------------------------------
----------------- Decorator -----------------
---------------------------------------------
--------------- Type Verifier ---------------
---------------------------------------------
------------- Intermediate Code -------------
---------------------------------------------
------------ Register Allocation ------------
('lit 0 0', {0})
('sto 0 c', {})
('lod 0 c', {0})
('lit 0 1', {0, 1})
('opr 0 2', {0})
('sto 0 c', {})
('lit 0 10', {0})
('sto 0 n1', {})
('lod 0 n1', {0})
('lit 0 5', {0, 1})
('opr 0 2', {0})
('sto 0 n2', {})
-------------- Intel 8086 Code --------------
mov ax, 0
mov c, ax
mov ax, c
mov bx, 1
add ax, bx
mov c, ax
mov ax, 10
mov n1, ax
mov ax, n1
mov bx, 5
add ax, bx
mov n2, ax
---------------------------------------------


## **Teste 2**

In [15]:
test = """
          program pessoas_concordam ;
            var p1, p2, p3 : boolean ;
                nao, msg : char ;
                ar : array [1..2] of boolean; 
            function concordam ( p1, p2 : boolean ) : boolean ;
              begin
                if ( p1 and p2 ) then
                  concordam := true
                else
                  concordam := false
              end
            begin
              read ( p1, p2 ) ;
              p3 := concordam ;
              nao := "nao" ;
              msg := "concordam" ;
              if p3 then
                write ( msg )
              else
                write ( nao, msg )
            end
            .
          
"""
tree = parser.parse(lexer.lex(test))
print("--------------- Symbol Table ----------------")
ST = {}
tree.accept(SymbolTable())
for k,v in ST.items():
  print(k, v)
print("---------------------------------------------")
print("----------------- Decorator -----------------")
tree.accept(Decorator())
print("---------------------------------------------")
print("--------------- Type Verifier ---------------")
tree.accept(TypeVerifier())
print("---------------------------------------------")
print("------------- Intermediate Code -------------")
ic = IntermediateCode()
tree.accept(ic)
ic = ic.ic
print("---------------------------------------------")
print("------------ Register Allocation ------------")
ra = RegisterAllocation()
ra.liveness_analysis(ic)
ra.add_graph_edges()
for i in ra.ic_la:
  print(i)
print("-------------- Intel 8086 Code --------------")
intel_8086 = Intel8086Code()
intel_8086.color_graph(ra.graph)
intel_8086.generate_code(ic)
for c in intel_8086.code:
  print(c)
print("---------------------------------------------")

--------------- Symbol Table ----------------
p1 boolean
p2 boolean
p3 boolean
nao char
msg char
ar array of boolean
concordam boolean
---------------------------------------------
----------------- Decorator -----------------
---------------------------------------------
--------------- Type Verifier ---------------
---------------------------------------------
------------- Intermediate Code -------------
---------------------------------------------
------------ Register Allocation ------------
-------------- Intel 8086 Code --------------
---------------------------------------------


## **Teste 3**

In [16]:
test = """
          program media_final ;
            var a, b, f, g, mf : integer ;
            function calcular_mf ( a, b, f, g : integer ) : integer ;
              var p1, p2, n1, n2 : integer ;
              begin
                p1 := 3 ;
                p2 := 7 ;
                a := a * p1 ;
                b := b * p2 ;
                f := f * p1 ;
                g := g * p2 ;
                n1 := (a + b) div 10 ;
                n2 := (f + g) div 10 ;
                calcular_mf := (n1 + n2) div 2 ;
              end
            begin
              a := 6 ;
              b := 10 ;
              f := 9 ;
              g := 10 ;
              mf := calcular_mf ;
              write ( mf ) ;
            end
            .
"""
tree = parser.parse(lexer.lex(test))
print("--------------- Symbol Table ----------------")
ST = {}
tree.accept(SymbolTable())
for k,v in ST.items():
  print(k, v)
print("---------------------------------------------")
print("----------------- Decorator -----------------")
tree.accept(Decorator())
print("---------------------------------------------")
print("--------------- Type Verifier ---------------")
tree.accept(TypeVerifier())
print("---------------------------------------------")
print("------------- Intermediate Code -------------")
ic = IntermediateCode()
tree.accept(ic)
ic = ic.ic
print("---------------------------------------------")
print("------------ Register Allocation ------------")
ra = RegisterAllocation()
ra.liveness_analysis(ic)
ra.add_graph_edges()
for i in ra.ic_la:
  print(i)
print("-------------- Intel 8086 Code --------------")
intel_8086 = Intel8086Code()
intel_8086.color_graph(ra.graph)
intel_8086.generate_code(ic)
for c in intel_8086.code:
  print(c)
print("---------------------------------------------")

--------------- Symbol Table ----------------
a integer
b integer
f integer
g integer
mf integer
calcular_mf integer
p1 integer
p2 integer
n1 integer
n2 integer
---------------------------------------------
----------------- Decorator -----------------
---------------------------------------------
--------------- Type Verifier ---------------
---------------------------------------------
------------- Intermediate Code -------------
---------------------------------------------
------------ Register Allocation ------------
('lit 0 3', {0})
('sto 0 p1', {})
('lit 0 7', {0})
('sto 0 p2', {})
('lod 0 a', {0})
('lod 0 p1', {0, 1})
('opr 0 4', {0})
('sto 0 a', {})
('lod 0 b', {0})
('lod 0 p2', {0, 1})
('opr 0 4', {0})
('sto 0 b', {})
('lod 0 f', {0})
('lod 0 p1', {0, 1})
('opr 0 4', {0})
('sto 0 f', {})
('lod 0 g', {0})
('lod 0 p2', {0, 1})
('opr 0 4', {0})
('sto 0 g', {})
('lod 0 a', {0})
('lod 0 b', {0, 1})
('opr 0 2', {0})
('lit 0 10', {0, 1})
('opr 0 5', {0})
('sto 0 n1', {})
('lod 0 f'